# Programme de calcul des dimensions de l'enceinte connectée

## Objectif

Le programme sert à résoudre l'équations non-linéaire décrivant les dimensions de l'enceinte connectée (modèle de Les Frères Poulain sur YouTube : https://youtu.be/ollr-PpCMAw). Pour cela on utilise la méthode de Newton.

## Licence 

Ce programme est sous licence CC-Zéro (licence libre non copyleft). Libre à vous d'utiliser ce code, de le modifier, de le partager ou même de le commercialiser.

## Auteur

Le code est écrit par Valentin Besse (valentinbesse@aumbox.net).

In [31]:
import math # importe le module mathematique.

# Declaration des variables de depart
volumeEnceinte = 8*10**6 # Volume de l'enceinte (mm3)
epaisseurBois = 20 # Epaisseur du bois (mm)
hauteurBassReflex = 115 # Hauteur du Bass Reflex (mmm)
diametreBassReflex = 70 # Diametre du Bass Reflex (mmm)
volumeBassReflex = 2*math.pi*diametreBassReflex*hauteurBassReflex # Volume du Bass Reflex à retiré (mm3)
volumeCorrige = volumeEnceinte + volumeBassReflex # Volume corrigé de l'intérieur de la boite.
ratioA = 2.6 # Ratio entre A et B
ratioC = 1.6 # Ratio entre C et B

print("""Volume enceinte : %e""" % volumeCorrige)

# Definition des fonctions nécessaires au calcul de la méthode de Newton.

# f(x) = 0
def dx(f, x):
    return abs(0-f(x))

# Methode de Newton
def newtons_method(f, df, x0, e):
    delta = dx(f, x0)
    while delta > e:
        x0 = x0 - f(x0)/df(x0)
        delta = dx(f, x0)
    print('La largeur A est : %1.0f' % (ratioA*x0), 'mm') # Largeur
    print('La profondeur B est : %1.0f' % x0, 'mm') # Profondeur
    print('La hauteur C est : %1.0f' % (ratioC*x0), 'mm') # Hauteur
    print('La largeur intérieure a est : %1.0f' % (ratioA*x0 - 2*epaisseurBois), 'mm') # Largeur intérieure
    print('La profondeur intérieure b est : %1.0f' % (x0 - 2*epaisseurBois), 'mm') # Profondeur intérieure
    print('La hauteur intérieure c est : %1.0f' % (ratioC*x0 - 2*epaisseurBois), 'mm') # Hauteur intérieure
    print("""Le recule du Bass Reflex d est  : %1.0f""" % (1/3*(x0 - 2*epaisseurBois)), 'mm') # Recule Bass Reflex
    print("""La profondeur intérieur corrigée est : %1.0f""" %(2/3*(x0 - 2*epaisseurBois)), """mm""") # Profondeur corrigée
    
    if 2/3*(x0 - 2*epaisseurBois) < hauteurBassReflex:
        print("""!!! ATTENTION !!!""")
        print("""La hauteur du Bass Reflex est supériuer à la profondeur.""")
    
# Fonction f(x)
def f(x):
    return volumeCorrige - 8/9*(x - 2*epaisseurBois)*(1.6*x - 2*epaisseurBois)*(2.6*x - 2*epaisseurBois)

# Derivee de la fonction f(x)
def df(x):
    return -8/9*((2.6*x - 2*epaisseurBois)*((1.6*x - 2*epaisseurBois) + 1.6*(x - 2*epaisseurBois)) + 2.6*(x - 2*epaisseurBois)*(1.6*x - 2*epaisseurBois))

x0s = [100]
for x0 in x0s:
     newtons_method(f, df, x0, 1e-5)

Volume enceinte : 8.050580e+06
La largeur A est : 408 mm
La profondeur B est : 157 mm
La hauteur C est : 251 mm
La largeur intérieure a est : 368 mm
La profondeur intérieure b est : 117 mm
La hauteur intérieure c est : 211 mm
Le recule du Bass Reflex d est  : 39 mm
La profondeur intérieur corrigée est : 78 mm
!!! ATTENTION !!!
La hauteur du Bass Reflex est supériuer à la profondeur.


## Auteur

Le code est écrit par Samuel Dupont

In [5]:
import scipy as sc
import scipy.linalg as scli

def newtons_method(volume,Ep,Rh = 2.6, Rl=1.6,K=0):
    # Allow to find the resulting inside/outside dimension of the desired vented box
    # volume -> desired volume
    # Ep -> width of the plank
    # Rh -> ratio of the length compared to the depht, default = 2.6
    # Rl -> ratio of the high compared to the depht, default = 1.6
    # K -> extra interior volume to remove
    sol = [1 , 1 , 1 , 1 ]
    for ii in range (1,100):
        J = sc.array(([8/9*sol[1]*sol[2], 8/9*sol[0]*sol[2], 8/9*sol[1]*sol[0],0],[0., 0, -1, Rl],[-1, 0, 0, Rh], [0, -1, 0, 1.]))
        J_inv = scli.inv(J)
        F =  sc.array((-volume +8/9*sol[0]*sol[1]*sol[2]-K , Rl*sol[3]-2*Ep-sol[2] , Rh*sol[3]-2*Ep-sol[0], sol[3]-sol[1]-2*Ep))
        sol = sol - J_inv.dot(F)
    sol = sc.concatenate( (sol[0:3], Rh*sol[3], sol[3], Rl*sol[3]),axis=None) 
    print('La largeur A est : %1.3f' % sol[3], 'm') # Largeur
    print('La profondeur B est : %1.3f' % sol[4], 'm') # Profondeur
    print('La hauteur C est : %1.3f' % sol[5], 'm') # Hauteur
    print('La largeur interieur a est : %1.3f' % sol[0], 'm') # Largeur
    print('La profondeur interieur b est : %1.3f' % sol[1], 'm') # Profondeur
    print('La hauteur interieur c est : %1.3f' % sol[2], 'm') # Hauteur
    return sol

V = 0.008
K=0.000
W=0.02
sol = newtons_method(V,W)
V_out = +8/9*sol[0]*sol[1]*sol[2]-K
print('Le volume resultant est : %1.1f' % (V_out*1e3), 'L \n') # Hauteur

V = 0.011
K=0.000
W=0.02
sol = newtons_method(V,W)
V_out = +8/9*sol[0]*sol[1]*sol[2]-K
print('Le volume resultant est : %1.1f' % (V_out*1e3), 'L \n') # Hauteur


La largeur A est : 0.407 m
La profondeur B est : 0.157 m
La hauteur C est : 0.250 m
La largeur interieur a est : 0.367 m
La profondeur interieur b est : 0.117 m
La hauteur interieur c est : 0.210 m
Le volume resultant est : 8.0 L 

La largeur A est : 0.445 m
La profondeur B est : 0.171 m
La hauteur C est : 0.274 m
La largeur interieur a est : 0.405 m
La profondeur interieur b est : 0.131 m
La hauteur interieur c est : 0.234 m
Le volume resultant est : 11.0 L 

